In [2]:
import numpy as np
from scipy import signal


data1 = np.load("./datasets/Gabi.npy")
data2 = np.load("./datasets/Jesse.npy")

data1.shape, data2.shape

((28, 1000, 4), (28, 1000, 4))

In [3]:
# Correção número de eletrodos e unificação

# os dois primeiros eletrodos são funcionais
data1 = data1[:,:,0:2]
data2 = data2[:,:,0:2]

data = np.array([data1, data2])

print(data.shape)

# colocando os dados na dimensão final
data = data.transpose(0, 1, 3, 2)
data.shape

(2, 28, 1000, 2)


(2, 28, 2, 1000)

In [4]:
# Funções para aplicação dos filtros temporais

def butter_bandpass(data, lowcut, highcut, fs=512, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)


def butter_notch(data, cutoff, var=1, fs=512, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

*Tarefa 1*: Realizem a aplicaçao dos filtros para toda a base.


In [5]:
# Aplicação dos filtros


# Mostrar até  (5 - 50hz)

# aplicação dos filtros
data_filtered = butter_notch(data, 60)
data_filtered = butter_bandpass(data_filtered, 5, 50)
data_filtered.shape

(2, 28, 2, 1000)

In [6]:
# # juntando participantes (apenas para demonstração)
# data = data_filtered.reshape(data_filtered.shape[0] * data_filtered.shape[1], data_filtered.shape[2], data_filtered.shape[3])
# data.shape

data = data_filtered

In [7]:
# Aplicação da segmentação (dom. tempo e frequência)

# Alterar valores abaixo para melhorar resultado do SVM
# alterar 200 e alterar 800
# data = data[:, :, :, 200:800] 


step = 29 # qual passo a cada deslizamento
segment = 64
print('', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3, 4)

# Janelas do dado no domínio da frequência
_, _, chunks_freq = signal.stft(data, fs=200, nperseg=segment, noverlap=32)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes, ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes, ensaios, canais, janelas, linhas)')


 (2, 28, 2, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (2, 28, 33, 2, 64) - (classes, ensaios, canais, janelas, linhas)
Dominio da frequência:  (2, 28, 33, 2, 33) - (classes, ensaios, canais, janelas, linhas)


*Tarefa 2*: Separar os participantes e armazenar os dados em disco para execução das proximas tarefas


In [8]:
print(chunks_freq[0].shape, chunks_freq[0].shape)
print(chunks_time[1].shape, chunks_time[1].shape)

gabi_freq = chunks_freq[0]
gabi_time = chunks_time[0]

jesse_freq = chunks_freq[1]
jesse_time = chunks_time[1]

np.save("./datasets/all_freq", chunks_freq)
np.save("./datasets/all_time", chunks_time)


np.save("./datasets/gabi_freq", gabi_freq)
np.save("./datasets/gabi_time", gabi_time)

np.save("./datasets/jesse_freq", jesse_freq)
np.save("./datasets/jesse_time", jesse_time)

(28, 33, 2, 33) (28, 33, 2, 33)
(28, 33, 2, 64) (28, 33, 2, 64)
[[[[ 9.51340802e-01 -7.87743457e+00 -1.63732030e+01 ...  1.27433551e+01
     1.25171441e+01  1.25970534e+01]
   [ 5.25451737e-01 -1.28029477e+00 -2.59354226e+00 ...  4.34990863e+00
     5.12094766e+00  5.75210986e+00]]

  [[-1.63837051e+01 -1.16385856e+01 -6.97657342e+00 ... -1.09617080e+01
    -1.17613564e+01 -1.23717950e+01]
   [ 3.86862186e+00  3.06814564e+00  1.58294116e+00 ...  9.94372052e+00
     9.28651457e+00  7.02641836e+00]]

  [[ 1.59444447e+01  1.44557579e+01  1.33746106e+01 ... -2.96952816e+00
    -3.01710669e+00 -2.89799377e+00]
   [ 1.54065356e+00  2.50722472e+00  3.46265529e+00 ... -6.02250924e+00
    -5.06320936e+00 -3.81906986e+00]]

  ...

  [[-2.43769562e+00 -3.27523666e+00 -3.86204525e+00 ... -2.76817477e+00
    -1.33810748e+00 -1.17482950e+00]
   [-1.03483102e+01 -1.09704636e+01 -1.24196254e+01 ... -2.31505428e+01
    -4.34479990e+01 -5.80988489e+01]]

  [[ 1.51081036e+01  1.66605983e+01  1.86567711e+